In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt

import numpy as np
import subprocess
import os


import netCDF4 as nc
import csv
import toml
from tomlkit import dumps
from tomlkit import parse
from tomlkit import inline_table

#import yaml
from ruamel.yaml import YAML
yaml = YAML()
yaml.preserve_quotes = True  # Keep quotes if they exist
yaml.indent(mapping=2, sequence=4, offset=2)  # Preserve indentation
yaml.representer.add_representer(
    float, lambda dumper, data: dumper.represent_scalar("tag:yaml.org,2002:float", str(data))
)

with open("../config/masterconfig.yaml", "r") as file:
    config = yaml.load(file) or {}
AGNI_rootpath = config['AGNI_rootpath']
AGNI_outputpath = config['AGNI_outputpath']
SOCRATES_path = config['SOCRATES_path']
REPO_path = config['REPO_path']# the path of the LIFEredoxsurvey repo

In [ ]:
filepath = "../output/atmospheres_mole.csv"
mole_data = np.genfromtxt(filepath,skip_header=1,delimiter=',')[:,2:]
coords = np.genfromtxt(filepath,skip_header=1,delimiter=',')[:,:2]
titles = np.genfromtxt(filepath,dtype=str,delimiter=',')[0][2:]

In [ ]:
def input_volatiles(filepath,volatile_dict):

    # open toml file
    with open(filepath, "r") as f:
        toml_str = f.read()
    doc = parse(toml_str)

    # input volatile dictionary
    my_table = inline_table()
    for key, value in volatile_dict.items():
        my_table[key] = value
    my_table.trivia.trail = "\n"
    doc["composition"]["vmr_dict"] = my_table

    # write to toml file
    with open(filepath, "w") as f:
        f.write(dumps(doc))  # Use str() to serialize back to TOML

In [ ]:
def write_data(filepath,bandcenter,intensity):
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['bandcenter','intensity',])
        for i in range(len(bandcenter)):
            string = [str(bandcenter[i]),str(intensity[i])]
            writer.writerow(string)

for i in range(len(mole_data)):
    # insert atmosphere and run AGNI
    dictionary = dict(zip(titles, mole_data[i]))
    input_volatiles("../config/LIFE_spectra.toml",dictionary)
    prompt = "export RAD_DIR=" + SOCRATES_path + ";./agni.jl " + REPO_path + "/config/LIFE_spectra.toml"
    subprocess.run(prompt,cwd=AGNI_rootpath, shell=True)

    # read out emission data
    AGNI_out = AGNI_outputpath + "/atm.nc"
    ds = nc.Dataset(AGNI_out)
    bandmin_data = ds['bandmin'][:]
    bandmax_data = ds['bandmax'][:]
    bandcenter = (bandmin_data + bandmax_data)/2 * 10**9
    bandwidth = (bandmax_data - bandmin_data) * 10**9
    lw_data = ds['ba_U_LW'][:]
    sw_data = ds['ba_U_SW'][:]
    bandflux = (lw_data[1, :] + sw_data[1, :]) / bandwidth * 1000

    # save text file
    csvname = '../output/AGNI/AGNI_textfiles/AGNI_spectrum_' + str(coords[i][0]) + '_' + str(coords[i][1]) + '.csv'
    write_data(csvname, bandcenter,bandflux)

In [ ]:
# raster plot

custom_order = []
for i in range(1,11):
    #print(i)
    custom_order.append(np.arange(90+i,i-10,-10))
custom_order = np.array(custom_order).flatten()

data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]

mpl.rcParams['figure.dpi'] = 800
rasterplot = figure(figsize=(8,8))

for i in range(len(files)):
    image_path = data_path + "/" + files[i]
    bandcenter = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,0]
    bandflux = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,1]
    # bandflux_error = np.genfromtxt(image_path,skip_header=1,delimiter=",",dtype=list)[:,1]
    # bandflux = []
    # for j in range(len(bandflux_error)):
    #     bandflux.append(float(bandflux_error[j].decode('utf8')[1:-1]))
    frame = rasterplot.add_subplot(10,10,custom_order[i])
    frame.plot(bandcenter,bandflux,linewidth=0.25)
    #frame.set_xlim(max(1.0e-10,min(bandcenter)), min(max(bandcenter), 70000.0))
    #frame.set_ylim(max(1.0e-10,min(bandflux)) / 2, max(bandflux) * 2)
    frame.set_xscale('log')
    frame.set_yscale('log')
    frame.set_yticklabels([])
    frame.set_xticklabels([])
    frame.set_xticks([])
    frame.set_yticks([])
    frame.minorticks_off()


    #frame.axis('off')

for j in range(10):
    left = 0.1 + j * 0.08
    ax = rasterplot.add_axes([left, 0.075, 0.1, 0.03])  # [left, bottom, width, height]
    ax.text(0.5, 0.5, str(round(coords[j*10][0],1)), ha='center', va='center')
    ax.axis('off')

for i in range(10):
    bottom = 0.1 + (9 - i) * 0.08
    ax = rasterplot.add_axes([0.1, bottom, 0.0, 0.07])
    ax.text(0.5, 0.5, str(round(coords[-i-1][1],1)), ha='center', va='center')
    ax.axis('off')

rasterplot.supxlabel('fo2 (IW)')
rasterplot.supylabel('H/C ratio')
rasterplot.show()
rasterplot.savefig('../output/figures/AGNI_rasterplot.png')
mpl.rcParams['figure.dpi'] = 300


In [ ]:
# save individual images

data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]

mpl.rcParams['figure.dpi'] = 300

for i in range(len(files)):
    image_path = data_path + "/" + files[i]
    bandcenter = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,0]
    bandflux = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,1]

    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bandcenter,bandflux)
    frame1.set_xlabel('wavelength (nm)')
    frame1.set_ylabel('spectral flux density (erg ps pcm2 pnm)')
    frame1.set_xscale('log')
    frame1.set_yscale('log')
    frame1.set_xlim(max(1.0e-10,min(bandcenter)), min(max(bandcenter), 70000.0))
    frame1.set_ylim(max(1.0e-10,min(bandflux)) / 2, max(bandflux) * 2)
    pngname = '../output/AGNI/AGNI_spectra/AGNI_spectrum_' + str(coords[i][0]) + '_' + str(coords[i][1]) + '.png'
    fig.savefig(pngname)